In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [170]:
f = '../output/stage1/Congo, Democratic Republic of the 2001 MICS.html'

In [171]:
html = BeautifulSoup(open(f).read(), 'lxml')

In [172]:
tables_bs = html.find_all('table')

In [173]:
len(tables_bs)

168

In [179]:
pd.read_html(tables_bs[np.random.randint(0, 168)].prettify(), thousands='')[0]

,0,1,2,3,4
0,Caractéristiques,Garçons,Filles,Total,Effectif d'enfants âgés de 6 ans
1,Milieu de résidence Urbain Rural Province K...,"35,9","30,3","33,0",531
2,"9,8","10,2","10,0",1.214,NaN
3,"50,7","33,3","41,9",169,NaN
4,"12,3","20,4","16,2",97,NaN
5,"16,2","26,8","21,9",196,NaN
6,"9,1","2,9","6,0",1n,NaN
7,"14,6","13,1","13 , 9",213,NaN
8,"4,9","2,8","3,7",160,NaN
9,"2,2","13,8","8,3",123,NaN


In [55]:
id_re = re.compile('[0-9S\.]*')
num_re = re.compile('^\(?[0-9]+[\.|,]?[0-9]*\)?')

In [148]:
def extract_tbs(t):
    
    if len(t.columns) < 3:
        print('not enough columns, skipping')
        return
    
    if not np.any(t[0].str.match('^[1-9\-]+')):
        return
    
    # fill names
    if t[0].hasnans:
        for i, v in t.iterrows():
            names = list()
            if pd.isnull(v[0]):
                n = t[0].ix[:i].last_valid_index()
                if not n:
                    continue
                while not (re.match('^[1-9S]', t[0].ix[n]) or t[0].ix[n].startswith('-')):
                    names.append(t[0].ix[n])
                    n = n - 1
                names.append(t[0].ix[n])
                names.reverse()
                t.ix[n:i, 0] = ' '.join(names)
    
    ids = []
    values = []
    names = []

    for i, row in t.iterrows():
        r = row.dropna().astype(str).values
    #     print(r[0].strip())
        if num_re.match(str(r[-1]).strip()):
            if len(r) > 2:
                if 'MDG' in r[1]:
                    name = r[2]
                else:
                    name = r[1]
            else:
                name = np.nan
            if re.match('^[1-9S\-]', str(r[0])):
                ks_ = str(r[0]).split(' ')
                vs_ = str(r[-1]).split(' ')
                ks = []
                
                for k in ks_:
                    if k == '':
                        continue
                    k = k.strip()
                    if k.strip() == '-':
                        ks.append(k)
                    else:
                        if re.match("[0-9S\.]+[a-z]?$", k):
                            ks.append(k)
                        else:
                            name = ' '.join(ks_[ks_.index(k):])
                            break
                    
                vs = [x for x in vs_ if x is not '']
                
                if len(ks) > len(vs):
                    print('warning: length not match:')
                    print(ks)
                    print(vs)
                    while len(ks) > len(vs):
                        vs.append(vs[-1])
                    
                if len(vs) > len(ks):
                    print('warning: length not match:')
                    print(ks)
                    print(vs)
                    while len(vs) > len(ks):
                        ks.append(ks[-1])
                    
                [ids.append(x) for x in ks]
                [names.append(name) for _ in ks]
                [values.append(x) for x in vs]
            else:
                ids.append(str(r[0]))
                values.append(str(r[-1]))
                names.append(name)
        else:
#             print('not num: ', r[-1])
            pass
    assert len(ids) == len(values)
    return pd.DataFrame({'id': ids, 'name': names, 'value': values})

In [149]:
extract_tbs(pd.read_html(tables_bs[12].prettify(), thousands='')[0])

['3.3', '3.5', '3.6']
['83.4']


,id,name,value
0,3.1,Tuberculosis immunization coverage,97.6
1,3.2,Polio immunization coverage,83.1
2,3.3,Pentavalent immunization coverage,83.4
3,3.5,Pentavalent immunization coverage,83.4
4,3.6,Pentavalent immunization coverage,83.4
5,3.4,Measles immunization coverage,90.2
6,3.8,Full immunization coverage,77.5


In [165]:
dfs = []

for i in range(6, 43):
    try:
        dfs.append(extract_tbs(pd.read_html(tables_bs[i].prettify(), thousands='')[0]))
    except:
        print(i)
        raise

# dfs = [extract_tbs(pd.read_html(tables_bs[x].prettify())[0]) for x in list(range(7, 47))]

['2.1a']
['12,3', '3,3']
['2.17a']
['4,6', '6,8']
not enough columns, skipping
['7.1']
['83,9', '88,8']
['8.5']
['26,9', '0,4']
['8.6']
['16,1', '1,5']
['8.12']
['(2)', 'elle', 'néglige', 'les', 'enfants,', '(3)', 'elle', 'discute', 'ses', 'opinions,', '(4)']
['9.5']
['16,2', '10,9']
['9.6']
['18,0', '8,2']
['9.9']
['39,1', '34,4']
['9.10']
['13,7', '16,8']
['9.12']
['5,7', '31,6']
['9.13']
['42,1', '36,6']
['9.14']
['39,9', '58,7']
['9.15']
['47,1', '61,0']
not enough columns, skipping
['10.3']
['21,1', '43,0']
['11.2']
['86,4', '81,6']
['11.3']
['47,2', '44,3']


In [166]:
adf = pd.concat(dfs, ignore_index=True)

In [167]:
adf

,id,name,value
0,1.1,Taux de mortalité néonatale,21
1,1.2,OMD 4.2,36
2,1.3,Taux de mortalité post- néonatale,15
3,1.4,Taux de mortalité juvénile,17
4,1.5,OMD 4.1,52
5,2.1a,OMD 1.8 Prévalence de l’insuffisance Pourcent...,"12,3"
6,2.1a,OMD 1.8 Prévalence de l’insuffisance Pourcent...,"3,3"
7,2.2a,Prévalence du retard de croissance Modérée et...,"21,2"
8,2.2b,Prévalence du retard de croissance Modérée et...,"8,2"
9,2.3a,Prévalence de l’émaciation Modérée et Sévère ...,"8,2"


In [169]:
adf.to_csv('../output/stage2/Congo_2014_15_French.csv', index=False)

In [220]:
if re.match("^[0-9S\.]+", 'viii'):
    print(':P')

In [147]:
re.match("[1-9S\.]+[a-z]?$", '2.10')

In [128]:
a = pd.read_html(tables_bs[47].prettify(), thousands='')[0]

In [132]:
np.any(a[0].str.match('^[0-9S\-]'))

True

In [137]:
'12'.split(' ')

['12']